In [2]:
import pandas as pd
from pulp import *

'''
For picking players, we also have several constraints. Those constraints are:
- A team consists of 15 players. In specific, a team has two goalkeepers, five defenders, five midfielders, and three forward players.
- The budget given for a team is only 100 million Pounds.
- We can pick only three players from a premier league team at maximum.

The optimisation workflow:

- Get the problem description.
- Formulate the mathematical program
- Solve the mathematical program
- Evaluate the result
- Finalize the result
'''

In [7]:
PATH = '/Users/stephenhill/Documents/Code Development/Data Science/fpl_team_project/data/'
pd.set_option("display.max_columns", None)

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv')
current_df.tail()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
24560,Wilfred Ndidi,MID,Leicester,0.0,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,16,0,0,0,0,38,0,70366,1,4,0.0,0,-202,22,224,48,True,0,38
24561,Matt Ritchie,DEF,Newcastle,0.9,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22T15:00:00Z,1,5,0,0,0,0,38,0,15999,2,1,0.0,1,143,396,253,49,False,0,38
24562,Nathan Redmond,MID,Southampton,3.4,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22T15:00:00Z,82,9,0,0,0,0,38,0,13856,1,4,0.0,2,455,683,228,59,False,0,38
24563,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,19,0,0,0,0,38,0,587,1,3,0.0,0,-2,0,2,45,True,0,38
24564,Ryan Fredericks,DEF,West Ham,0.2,0,0,0,0,0.0,415,373,0,0,0.0,0.0,2022-05-22T15:00:00Z,0,4,0,0,0,0,38,0,8856,1,3,0.0,0,36,103,67,44,False,0,38


In [11]:
gw_5 = current_df[current_df.GW == 5]
gw_5.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
2296,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,49,0,0,0.0,0.0,2021-09-19T13:00:00Z,0,19,0,0,0,0,5,0,7265,2,1,0.0,0,-624,252,876,50,False,0,5
2297,Keinan Davis,FWD,Aston Villa,0.0,0,0,0,0,0.0,49,41,0,0,0.0,0.0,2021-09-18T16:30:00Z,0,8,0,0,0,0,5,0,115711,0,3,0.0,0,-7359,9224,16583,45,True,0,5
2298,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,47,0,0,0.0,0.0,2021-09-18T14:00:00Z,0,15,0,0,0,0,5,0,1709,3,1,0.0,0,-267,0,267,45,False,0,5
2299,James Ward-Prowse,MID,Southampton,3.0,0,0,13,1,12.2,341,45,0,0,2.6,13.6,2021-09-18T14:00:00Z,90,12,0,0,0,0,5,0,268540,0,0,0.0,3,-20295,12541,32836,64,False,0,5
2300,Bruno Miguel Borges Fernandes,MID,Man Utd,3.2,0,0,17,0,48.8,277,49,1,0,10.5,18.0,2021-09-19T13:00:00Z,90,19,0,0,0,0,5,0,2797372,2,1,38.0,2,-552801,55156,607957,120,False,0,5


In [13]:
# isolate the columns we are interested in - name, team, position, total points and value

data = gw_5[['name', 'team', 'position', 'total_points', 'value']]
data.head()

,name,team,position,total_points,value
2296,Eric Bailly,Man Utd,DEF,0,50
2297,Keinan Davis,Aston Villa,FWD,0,45
2298,Ayotomiwa Dele-Bashiru,Watford,MID,0,45
2299,James Ward-Prowse,Southampton,MID,3,64
2300,Bruno Miguel Borges Fernandes,Man Utd,MID,2,120


In [15]:
'''
The next step is to initialize several variables. 
As I’ve mentioned before, we took several columns from the table. 
The reason for that is because of two things, the objective, and the constraints.
'''

# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()
BUDGET = 1000
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = [data.name[i] for i in data.index]
teams = [data.team[i] for i in data.index]
positions = [data.position[i] for i in data.index]
prices = [data.value[i] for i in data.index]
points = [data.total_points[i] for i in data.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in data.index]

In [16]:
'''
Initialize the problem
----------------------
Now we have the variables that we need. 
The next step is to initialize the LpProblem object that contains our objective and constraints.
We set parameters like the name of the problem and an object to determine the goal of our problem. 
Because we want to maximize the number of points, we set the LpMaximize as the object’s parameter.
'''

# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

/Users/stephenhill/opt/anaconda3/envs/fpl_env/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [17]:
'''
Define the objective
--------------------
After we initialize the problem, the next step is to define the objective. 
The objective of our problem is to maximize the number of points.
'''

# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(data))) # Objective

In [18]:
'''
Define the constraints
----------------------
Let’s recalled the constraints from the FPL:
- A team consists of 15 players. In specific, a team has two goalkeepers, five defenders, five midfielders, and three forward players.
- The budget given for a team is only 100 million Pounds.
- We can pick only three players from a premier league team at maximum.
Based on the constraints above, we create the expression that mathematically describes the constraints.
'''

# Build the constraints

# Budget Limit
prob += lpSum(players[i] * data.value[data.index[i]] for i in range(len(data))) <= BUDGET

for pos in POS:
    # Position Limit
    prob += lpSum(players[i] for i in range(len(data)) if positions[i] == pos) <= pos_available[pos]
    
for club in CLUBS:
    # Club Limit
    prob += lpSum(players[i] for i in range(len(data)) if teams[i] == club) <= 3

In [19]:
'''
Solve the problem
-----------------
We have the variables and mathematical expressions that we need.
'''

# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/stephenhill/opt/anaconda3/envs/fpl_env/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/8w/26d3d16j5rl45hcb3gt3x3pw0000gn/T/5780fdc9536f4085a71263c872473a61-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/8w/26d3d16j5rl45hcb3gt3x3pw0000gn/T/5780fdc9536f4085a71263c872473a61-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3334 RHS
At line 3360 BOUNDS
At line 3967 ENDATA
Problem MODEL has 25 rows, 606 columns and 1818 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 173 - 0.00 seconds
Cgl0004I processed model has 25 rows, 245 columns (245 integer (223 of which binary)) and 735 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution fou

1

In [20]:
'''
Retrieve the list of players
----------------------------
After the program solves the problem, we can retrieve the player’s names that adhere to the constraints.
'''

for v in prob.variables():
    if v.varValue != 0:
        name = data.name[int(v.name.split("_")[1])]
        club = data.team[int(v.name.split("_")[1])]
        position = data.position[int(v.name.split("_")[1])]
        point = data.total_points[int(v.name.split("_")[1])]
        price = data.value[int(v.name.split("_")[1])]
        print(name, position, club, point, price, sep=" | ")

Joshua King | FWD | Watford | 10 | 55
Aaron Ramsdale | GK | Arsenal | 9 | 45
Mohamed Salah | MID | Liverpool | 12 | 125
Virgil van Dijk | DEF | Liverpool | 12 | 65
Saïd Benrahma | MID | West Ham | 10 | 64
Antonio Rüdiger | DEF | Chelsea | 14 | 55
Ismaila Sarr | MID | Watford | 15 | 60
Marcos Alonso | DEF | Chelsea | 10 | 57
Matthew Cash | DEF | Aston Villa | 15 | 50
Ivan Toney | FWD | Brentford | 12 | 63
David de Gea | GK | Man Utd | 10 | 50
Danny Welbeck | FWD | Brighton | 9 | 60
Martin Ødegaard | MID | Arsenal | 11 | 55
Leon Bailey | MID | Aston Villa | 9 | 64
Thiago Emiliano da Silva | DEF | Chelsea | 15 | 54


In [21]:
'''
Retrieve the expected points and the total costs
------------------------------------------------
Now you have the team squad that suits the constraints from FPL. 
What if we want to know the expected points and the used budget?
From the prob, we can retrieve the objective and the constraints equation. 
We can calculate the total points and the price value by evaluating the equations.
'''

score = str(prob.objective)
constraint = [str(const) for const in prob.constraints.values()][0]
for v in prob.variables():
    score = score.replace(v.name, str(v.varValue))
    constraint = constraint.replace(v.name, str(v.varValue))

score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

print("Constraint: ")
print(constraint_pretty + " = " + str(eval(constraint_pretty)))
print()
print("Score: ")
print(score_pretty + " = " + str(eval(score_pretty)))

Constraint: 
55*1.0 + 45*1.0 + 125*1.0 + 65*1.0 + 64*1.0 + 55*1.0 + 60*1.0 + 57*1.0 + 50*1.0 + 63*1.0 + 50*1.0 + 60*1.0 + 55*1.0 + 64*1.0 + 54*1.0 = 922.0

Score: 
10*1.0 + 9*1.0 + 12*1.0 + 12*1.0 + 10*1.0 + 14*1.0 + 15*1.0 + 10*1.0 + 15*1.0 + 12*1.0 + 10*1.0 + 9*1.0 + 11*1.0 + 9*1.0 + 15*1.0 = 173.0
